In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from os import walk, remove
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'

%matplotlib inline

files = next(walk('./input'), (None, None, []))[2]  # [] if no file
outputFiles = next(walk('./output'), (None, None, []))[2]  # [] if no file
for outputFile in outputFiles:
    remove('./output/' + outputFile)


def filterFunction(cnt, image):
    x, y, w, h = cv2.boundingRect(cnt)
    if h > (image.shape[0] / 2) and w < (image.shape[1] / 5) and x > (3 * image.shape[1] / 4):
        return True
    return False

def filterFunction2(cnt, image):
    x, y, w, h = cv2.boundingRect(cnt)
    if h > (image.shape[0] / 30):
        return True
    return False

def readText(image):
    image = 255 - image
    imgplot = plt.imshow(image, cmap='gray')
    plt.show()
    data = pytesseract.image_to_string(image, lang='eng', config='--oem 3 --psm 6 -c tessedit_char_whitelist=0123456789')
    return data

def readText2(image):
    image = 255 - image
    imgplot = plt.imshow(image, cmap='gray')
    plt.show()
    data = pytesseract.image_to_string(image, lang='eng', config='--oem 3 --psm 6 -c tessedit_char_whitelist=0123456789')
    return data


def getLvlCoords(imgs):
    lvlCoords = []
    for img in imgs:
        img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        image_enhanced = cv2.equalizeHist(img_gray)
        thresh = 225
        im_bw = cv2.threshold(img_gray, thresh, 255, cv2.THRESH_BINARY)[1]
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
        dilate = cv2.dilate(im_bw, kernel, iterations=3)
        
        cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        cnts = filter(lambda item: cv2.contourArea(item) > 300, cnts)
        cnts = sorted(cnts, key=lambda item1: item1[0][0][0])

        if(len(cnts) > 2):
            for i in range(0, 2):
                x, y, w, h = cv2.boundingRect(cnts[i])
                x2, y2, w2, h2 = cv2.boundingRect(cnts[i + 1])
                if(x2 - (x + w) > 50):
                    break
                    
        elif(len(cnts) == 1):
            x, y, w, h = cv2.boundingRect(cnts[0])


        else:
            continue
        
        if(x > img.shape[1] / 7):
            continue
        
        if((w / h) > 1.25):
            x = int((x + w) - h * 1.25)
            w = int(h * 1.25)

            
        lvlCoords.append([x, y, w, h])
        
    i = 0
    lvlCoords2 = []
    for x, y, w, h in lvlCoords:
        for y in range(i + 1, len(lvlCoords)):
            x2, y2, w2, h2 = lvlCoords[y]
            if(abs((x + w) - (x2 + w2)) < 30):
                lvlCoords2.append([x, y, w, h])
                break
        i += 1
    x, y, w, h = 0, 0, 0, 0
    for x2, y2, w2, h2 in lvlCoords2:
        x += x2
        y += y2
        w += w2
        h += h2
                                     
    if(len(lvlCoords2) != 0):
        x /= len(lvlCoords2)
        y /= len(lvlCoords2)
        w /= len(lvlCoords2)
        h /= len(lvlCoords2)
        
    
    return int(x), int(y), int(w), int(h)


def getUsernameCoords(imgs, x_lvl):
    lvlCoords = []
    for img in imgs:
        img = img[:, x_lvl:]
        img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        image_enhanced = cv2.equalizeHist(img_gray)
        thresh = 225
        im_bw = cv2.threshold(img_gray, thresh, 255, cv2.THRESH_BINARY)[1]
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
        dilate = cv2.dilate(im_bw, kernel, iterations=3)
        
        cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        cnts = filter(lambda item: cv2.contourArea(item) > 300, cnts)
        cnts = sorted(cnts, key=lambda item1: item1[0][0][0])

        if(len(cnts) > 2):
            for i in range(0, 2):
                x, y, w, h = cv2.boundingRect(cnts[i])
                x2, y2, w2, h2 = cv2.boundingRect(cnts[i + 1])
                if(x2 - (x + w) > 50):
                    break
                    
        elif(len(cnts) == 1):
            x, y, w, h = cv2.boundingRect(cnts[0])


        else:
            continue
        
        if(x > img.shape[1] / 3):
            continue
            
        lvlCoords.append([x, y, w, h])
    i = 0
    lvlCoords2 = []
    for x, y, w, h in lvlCoords:
        for y in range(i + 1, len(lvlCoords)):
            x2, y2, w2, h2 = lvlCoords[y]
            if(abs(x - x2) < 30):
                lvlCoords2.append([x, y, w, h])
                break
        i += 1
    x, y, w, h = 0, 0, 0, 0
    for x2, y2, w2, h2 in lvlCoords2:
        x += x2
        y += y2
        w = w2 if w2 > w else w
        h += h2
                                     
    if(len(lvlCoords2) != 0):
        x /= len(lvlCoords2)
        y /= len(lvlCoords2)
        h /= len(lvlCoords2)
        
    
    return int(x), int(y), int(w), int(h)

def getGoldCoords(imgs):
    lvlCoords = []
    for img in imgs:
        img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        image_enhanced = cv2.equalizeHist(img_gray)
        thresh = 225
        im_bw = cv2.threshold(img_gray, thresh, 255, cv2.THRESH_BINARY)[1]
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
        dilate = cv2.dilate(im_bw, kernel, iterations=3)
        
        cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        cnts = filter(lambda item: cv2.contourArea(item) > 300, cnts)
        cnts = sorted(cnts, key=lambda item1: item1[0][0][0], reverse=True)

        if(len(cnts) > 1):
            x, y, w, h = cv2.boundingRect(cnts[0])


        else:
            continue
        
        if(x < 3 * img.shape[1] / 4):
            continue
            
        lvlCoords.append([x, y, w, h])
        
    i = 0
    lvlCoords2 = []
    for x, y, w, h in lvlCoords:
        for y in range(i + 1, len(lvlCoords)):
            x2, y2, w2, h2 = lvlCoords[y]
            if(abs((x + w) - (x2 + w2)) < 30):
                lvlCoords2.append([x, y, w, h])
                break
        i += 1
    x, y, w, h = 0, 0, 0, 0
    for x2, y2, w2, h2 in lvlCoords2:
        x = x2 if x2 > x else x
        w = w2 if w2 > w else w
                                         
    return int(x), int(y), int(w), int(h)


def getDmgCoords(imgs, x_gold):
    lvlCoords = []
    for img in imgs:
        img = img[:, :x_gold]
        img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        image_enhanced = cv2.equalizeHist(img_gray)
        thresh = 225
        im_bw = cv2.threshold(img_gray, thresh, 255, cv2.THRESH_BINARY)[1]
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
        dilate = cv2.dilate(im_bw, kernel, iterations=3)
        
        cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        cnts = filter(lambda item: cv2.contourArea(item) > 300, cnts)
        cnts = sorted(cnts, key=lambda item1: item1[0][0][0], reverse=True)

        if(len(cnts) > 1):
            x, y, w, h = cv2.boundingRect(cnts[0])


        else:
            continue
        
        if(x < 3 * img.shape[1] / 4):
            continue
            
        lvlCoords.append([x, y, w, h])
        
    i = 0
    lvlCoords2 = []
    for x, y, w, h in lvlCoords:
        for y in range(i + 1, len(lvlCoords)):
            x2, y2, w2, h2 = lvlCoords[y]
            if(abs((x + w) - (x2 + w2)) < 30):
                lvlCoords2.append([x, y, w, h])
                break
        i += 1
    x, y, w, h = 0, 0, 0, 0
    for x2, y2, w2, h2 in lvlCoords2:
        x = x2 if x2 > x else x
        w = w2 if w2 > w else w
                                         
    return int(x), int(y), int(w), int(h)


for path in files:
    image = cv2.imread(f'./input/{path}')
    # cv2.imwrite(f"./output/{path.split('.')[0]}.png", image)  
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    im_gray = cv2.imread(f'./input/{path}', cv2.IMREAD_GRAYSCALE)
    image_enhanced = cv2.equalizeHist(im_gray)
    thresh = 200
    im_bw = cv2.threshold(image_enhanced, thresh, 255, cv2.THRESH_BINARY)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 3))
    dilate = cv2.dilate(im_bw, kernel, iterations=25)
    # cv2.imwrite(f"./output/{path.split('.')[0]}_dilatedY.png", dilate) 

    # troba la linea vermella de la esquerra per poder retallar
    lower_red = np.array([170,50,50])
    upper_red = np.array([360,250,250])
    img_hsl = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(img_hsl, lower_red, upper_red)
    res = cv2.bitwise_and(img_hsl,img_hsl, mask = mask)
    # Detect horizontal lines
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,25))
    detect_vertical = cv2.morphologyEx(mask, cv2.MORPH_OPEN, vertical_kernel, iterations=2)
    cnts = cv2.findContours(detect_vertical, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    # sort by lenght
    cnts = filter(lambda item: cv2.contourArea(item) > 100, cnts)
    cnts = sorted(cnts, key=lambda item1: item1[0][0][0])
    x, y, w, h = cv2.boundingRect(cnts[0])
    image = image[:, x + w:]
    # cv2.imwrite(f"./output/{path.split('.')[0]}_mask.png", image) 

    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = filter(lambda item: filterFunction(item, im_bw), cnts)
    cnts = sorted(cnts, key=lambda item1: item1[0][0][0], reverse=True)
    if(len(cnts) > 0):
        x, y, w, h = cv2.boundingRect(cnts[0])
        # cv2.imwrite(f"./output/{path.split('.')[0]}_cropped.png", image[:, 0:x - 10])
        image = image[:, 0:x - 10] #image without friend list
        im_gray = im_gray[:, 0:x - 10]
        image_enhanced = cv2.equalizeHist(im_gray)
    
    thresh = 250
    im_bw = cv2.threshold(image_enhanced, thresh, 255, cv2.THRESH_BINARY)[1]

    # cv2.imwrite(f"./output/{path.split('.')[0]}_black.png", im_bw) 
    # image ja no te la part dels amics, ara treurem la part de dalt i baix

    # Dilate to combine adjacent text contours
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 1))
    dilate = cv2.dilate(im_bw, kernel, iterations=100)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 3))
    dilate = cv2.dilate(dilate, kernel, iterations=1)
    # cv2.imwrite(f"./output/{path.split('.')[0]}_dilatedH.png", dilate)  

    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = filter(lambda item: filterFunction2(item, im_bw), cnts)
    cnts = sorted(cnts, key=lambda item1: item1[0][0][1])
    # for cnt in cnts:
    #     cv2.drawContours(image, [cnt], 0, (255,255,0), 3)
    # cv2.imwrite(f"./output/{path.split('.')[0]}_cnts.png", image)  

    top = None
    bottom = None
    if(len(cnts) > 0):
        foundTop = False
        foundBottom = False
        cnts2 = cnts[:]
        victoryImg = None
        while not foundTop:
            x, y, w, h = cv2.boundingRect(cnts2[0])
            x2, y2, w2, h2 = cv2.boundingRect(cnts2[1])
            if(y2 - (y + h) > image.shape[0] / 20):
                victoryImg = image[0:y + h, :]
                # cv2.imwrite(f"./output/{path.split('.')[0]}_victory.png", victoryImg)
                top = y2
                foundTop = True
            elif(y > image.shape[0] / 10): 
                top = y
                foundTop = True
            else:
                cnts2.pop(0)
            if(len(cnts2) < 2):
                top = 0
                foundTop = True

        cnts2 = cnts[:]
        while not foundBottom:
            x, y, w, h = cv2.boundingRect(cnts2[-1])
            x2, y2, w2, h2 = cv2.boundingRect(cnts2[-2])
            if(y - (y2 + h2) > image.shape[0] / 20):
                bottom = y2 + h2
                foundBottom = True
            elif(y > image.shape[0] / 2):
                bottom = y + h
                foundBottom = True
            else:
                cnts2.pop(-1)
            if(len(cnts2) < 2):
                bottom = image.shape[0]
                foundBottom = True
        
        image = image[top:bottom, :]
        im_gray = im_gray[top:bottom, :]
        # cv2.imwrite(f"./output/{path.split('.')[0]}_final_cropped.png", image)

        # LA IMATGE JA ESTÀ RETALLADA I NOMES HI HA LA INFO UTIL (a vegades es queden els amics, retallarem la imatge amb un format estandard)
        height = image.shape[0]
        width = image.shape[1]
        if(width / height > 2.25):
            image = image[:, 0: int(2.25 * height)]
            im_gray = im_gray[:, 0: int(2.25 * height)]
        # cv2.imwrite(f"./output/{path.split('.')[0]}_final_cropped.png", image)
        image_enhanced = cv2.equalizeHist(im_gray)
        thresh = 250
        im_bw = cv2.threshold(image_enhanced, thresh, 255, cv2.THRESH_BINARY)[1]
        # cv2.imwrite(f"./output/{path.split('.')[0]}_black2.png", im_bw)


        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 1))
        dilate = cv2.dilate(im_bw, kernel, iterations=100)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 3))
        dilate = cv2.dilate(dilate, kernel, iterations=1)

        cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        cnts = filter(lambda item: filterFunction2(item, im_bw), cnts)
        cnts = sorted(cnts, key=lambda item1: item1[0][0][1])
        # for cnt in cnts:
        #     cv2.drawContours(image, [cnt], 0, (255,255,0), 3)
        # cv2.imwrite(f"./output/{path.split('.')[0]}_cnts.png", image) 

        imgs = []
        if(len(cnts) == 10):
            for cnt in cnts:
                x, y, w, h = cv2.boundingRect(cnt)
                imgs.append(image[y:y + h + 5, :])
        else:
            x, y, w, h = cv2.boundingRect(cnts[0])
            x2, y2, w2, h2 = cv2.boundingRect(cnts[-1])

            i = 0
            maxSeparation = 0
            cntTop = None
            cntBottom = None
            for i in range(0, len(cnts) - 1):
                x3, y3, w3, h3 = cv2.boundingRect(cnts[i])
                x4, y4, w4, h4 = cv2.boundingRect(cnts[i + 1])
                if(y4 - (y3 + h3) > maxSeparation):
                    maxSeparation = y4 - (y3 + h3)
                    cntTop = cnts[i]
                    cntBottom = cnts[i + 1]
                

            if(cntTop is not None and cntBottom is not None):
                x4, y4, w4, h4 = cv2.boundingRect(cntBottom)
                height = (y2 + h2 - y4) / 5 + 3
                for i in range(0, 10):
                    y5 = y2 - height * i
                    if(i > 4):
                        y5 -= maxSeparation / 2
                    imgs.append(image[int(y5):int(y5 + height), :])
                    imgs.reverse()

        i = 0

        x, y, w, h = getLvlCoords(imgs)
        x2, y2, w2, h2 = getUsernameCoords(imgs, x + w)
        x2 += (x + w)
        x3, y3, w3, h3 = getGoldCoords(imgs)
        x4, y4, w4, h4 = getDmgCoords(imgs, x3 - 5)
        x5, y5, w5, h5 = getDmgCoords(imgs, x4 - 5)


        for img in imgs:
                
            # for cnt in cnts:
                #draw contour
                # cv2.drawContours(img, [cnt], 0, (255,255,0), 3)
            # cv2.imwrite(f"./output/{path.split('.')[0]}_level.png", img)
            lvl = img[:, x:x+w] 
            img_gray = cv2.cvtColor(lvl, cv2.COLOR_RGB2GRAY)
            thresh = 200
            im_bw = cv2.threshold(img_gray, thresh, 255, cv2.THRESH_BINARY)[1]
            

            #print(f'{readText(lvl)} - {path.split(".")[0]}')
            #cv2.imwrite(f"/kaggle/working/{path.split('.')[0]}_level.png", lvl)
            imgplot = plt.imshow(im_bw)
            plt.show()
            print(readText(im_bw))
            
            
            # username = img_gray[:, x2:x2+w2]
            # imgplot = plt.imshow(username)
            # plt.show()
            # print(readText(username))

            gold = img[0:int(img.shape[0] * 0.55), (x3 - 5):(x3 + w3)]
            gold_gray=cv2.cvtColor(gold,cv2.COLOR_RGB2GRAY)
            out_binary=cv2.threshold(gold_gray, 0, 255, cv2.THRESH_OTSU )[1] 
            img2 = cv2.copyMakeBorder(out_binary, 5,5,5,5, borderType=cv2.BORDER_CONSTANT, value=0)

           
            imgplot = plt.imshow(img2, cmap='gray')
            plt.show()
            print(readText2(img2))

            # dmg = img_gray[0:int(img.shape[0] * 0.55), (x4 - 5):(x4 + w4)]
            # imgplot = plt.imshow(dmg)
            # plt.show()
            # print(readText(dmg))

            # kda = img_gray[0:int(img.shape[0] * 0.55), (x5 - 5):(x5 + w5)]
            # imgplot = plt.imshow(kda)
            # plt.show()
            # print(readText(kda))





        